## Disclaimer

**Environment:** This notebook has been tested with:

- Julia v1.11.x
- JuMP.jl v1.26.x
- HiGHS v1.18.1

**Author:** Xavier Gandibleux, Nantes (France)


----

## Part 3: Multi-Objective Optimization

Given the bi-objective 01 Unidimensional Knapsack Problem (bi-01UKP) formulated by 
$$z=\max\big\{(p^1x,\ p^2x) \mid wx \le c, \ x\in\{0,1\}^n\big\}$$

and the numerical instance corresponding to 
$$n=5$$
$$p^1=(6, \ 4, \ 4, \ 4, \ 3)$$
$$p^2=(12, 10, \ 5, \ 3, \ 1)$$
$$w=(\ 8, \ 6, \ 4, \ 3, \ 2)$$
$$c=15$$

answer to the following questions:

1. using the package `JuMP`, implement an implicit formulation of the bi-01UKP and display the model.
2. using the package `MultiObjectiveAlgorithms`, compute and display $X_E$ and $Y_N$, a set of efficient solutions and the set of non-dominated points.
3. using the package `Plots`, display $Y_N$ in the objective space $Y$


## Answers:

----
### 1. using the package `JuMP`, implement an implicit formulation of the bi-01UKP and display the model.

#### declare the package(s) to use:

In [1]:
using JuMP, HiGHS
import MultiObjectiveAlgorithms as MOA

<br>

#### setup the instance to solve

In [2]:
p1 = [  6,  4, 4, 4, 3 ] # profit 1
p2 = [ 12, 10, 5, 3, 1 ] # profit 2
w  = [  8,  6, 4, 3, 2 ] # weight
c  = 15                                        # capacity
n  = length(p1)           # number of items

5

<br>

#### setup the formulation

In [3]:
bi01UKP = Model( )

@variable(bi01UKP, x[1:n], Bin)

@expression(bi01UKP, objective1, sum(p1[i] * x[i] for i in 1:n))
@expression(bi01UKP, objective2, sum(p2[i] * x[i] for i in 1:n))
@objective(bi01UKP, Max, [objective1, objective2])

@constraint(bi01UKP, sum(w[i] * x[i] for i in 1:n) ≤ c)

print(bi01UKP)

<br>

---- 
### 2. using the package `MultiObjectiveAlgorithms`, compute $Y_N$, the set of non-dominated points.

#### setup the MIP solver to use
<ins>Hints:</ins> see `set_optimizer()`

In [4]:
set_optimizer(bi01UKP, () -> MOA.Optimizer(HiGHS.Optimizer))

<br>

#### setup the algorithm to use
<ins>Hints:</ins> see `set_attribute()`

In [5]:
set_attribute(bi01UKP, MOA.Algorithm(), MOA.EpsilonConstraint())

<br>

#### solve the problem

In [6]:
set_silent(bi01UKP)
optimize!(bi01UKP)
@assert is_solved_and_feasible(bi01UKP)

<br>

#### query the number of points in $Y_N$
<ins>Hints:</ins> see `result_count()`

In [7]:
result_count(bi01UKP)

3

<br>

#### query the vector values of the 3rd point

In [8]:
objective_value(bi01UKP; result = 3)

2-element Vector{Float64}:
 15.0
 19.0

<br>

#### query the value for the first objective of the 3rd point

In [9]:
value(objective1; result = 3)

15.0

<br>

#### display $X_E$ and $Y_N$

In [10]:
for i in 1:result_count(bi01UKP)
    @assert is_solved_and_feasible(bi01UKP; result = i)
    print(i, ": x = ", round.(Int, value.(x; result = i)), " | ")
    println("z = ", round.(Int, objective_value(bi01UKP; result = i)))
end

1: x = [1, 1, 0, 0, 0] | z = [10, 22]
2: x = [1, 0, 1, 1, 0] | z = [14, 20]
3: x = [0, 1, 1, 1, 1] | z = [15, 19]


<br>

----
### 3. using the package `Plots`, display $Y_N$ in the objective space $Y$

#### declare the package(s) to use:

In [ ]:
using Plots

<br> 

#### plot $Y_N$ in $Y$:

In [ ]:
Plots.scatter(
      [value(objective1; result = i) for i in 1:result_count(bi01UKP)],
      [value(objective2; result = i) for i in 1:result_count(bi01UKP)];
      xlabel = "\$z_1(x)\$",
      ylabel = "\$z_2(x)\$",
      title  = "Knapsack: Objective space \$Y\$",
      label  = "",
      xlims  = (5, 25),
      ylims  = (5, 25),
      aspect_ratio=:equal
)

for i in 1:result_count(bi01UKP)
    y = objective_value(bi01UKP; result = i)
    Plots.annotate!(y[1] - 0.75, y[2] - 0.75, (i, 8))
end

ideal_point = objective_bound(bi01UKP)
Plots.scatter!([ideal_point[1]], [ideal_point[2]]; label = "Ideal point")